In [ ]:
# Import packages
from pypdf import PdfReader
import gradio as gr
from IPython.display import display, Markdown
from agents import Agent, Runner, trace, function_tool

In [ ]:
import os
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv(override=True)

endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o-mini")

# Initialize Azure OpenAI client with Entra ID authentication
token_provider = get_bearer_token_provider(
    DefaultAzureCredential(),
    "https://cognitiveservices.azure.com/.default"
)

openai = AzureOpenAI(
    azure_endpoint=endpoint,
    azure_ad_token_provider=token_provider,
    api_version="2025-01-01-preview",
)

In [ ]:
reader = PdfReader("files/WS2_Architecture_proposal.pdf")
architecture_proposal = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        architecture_proposal += text

In [ ]:
print(architecture_proposal)

In [ ]:
adr_template = """---
title: ADR Template
---

---
MADR template version: 3.0.0 
MADR template source: https://github.com/adr/madr
---

# ADR Template

## Title. 
*mandatory*  
The title assigns a {name} to the AD so that it can be identified and searched for easily. Ideally, it should convey the essence of the problem solved and the solution chosen.

## Metadata.
*mandatory*  

The metadata elements are:

| Attribute Name | Description | Format |mandatory optional|
| -------------- | ----------- | ------ |------|
| status | status of the architecture decision | proposed \\| accepted \\| rejected \\| deprecated \\| superseded by | mandatory |
| date | when the decision was last updated | {YYYY-MM-DD} | mandatory |
| deciders | lists everyone involved in the decision |  | mandatory |
| consulted | lists everyone whose opinions are sought and with whom there is a two-way communication |  | optional |
| informed | lists everyone who is kept up-to-date on progress in one-way communication |  | optional |

### NN additions  
*mandatory*

| Attribute Name | Description | Format |mandatory optional|
| -------------- | ----------- | ------ |------|
| Identifier | Unique identifier of the decision | < integer > | mandatory |
| Organizational Scope | Identification of the organizational scope to which the architecture decision applies. This scope comprises of one or more predefined organizational units. | NN Group \\| < country > \\| < BU > \\| < country >\\\\< BU > \\| < country >\\\\< BU >\\\\< LoB > | mandatory |
| Functional Scope | Identification of the functional scope to which the architecture decision applies. This scope comprises of one or more domains, subdomains or digital products, preferably predefined, but free text is acceptable as a minimal viable solution. | < domain > \\| < domain >\\\\< subdomain > \\| < domain >\\\\< subdomain >\\\\< digital product > | mandatory |

## Context and Problem Statement.  
*mandatory*  
Describes the context and problem statement in a few sentences. One may want to articulate the problem in form of a question or provide an illustrative story that invites to a conversation. Links to collaboration boards or issue management systems can go here too.

## Decision Drivers.  
*optional*  
Desired qualities, forces, faced concerns are identified here:

- {decision driver n}

## Considered Options.  
*mandatory*  
This section lists the alternatives (or choices) investigated:

- {title/name of option n}

The template recommends listing the chosen option first (as a project-wide convention). One needs to make sure to list options that can solve the given problem in the given context (as documented in Section “Context and Problem Statement”). They should do so on the same level of abstraction; a mistake we have seen in practice is that a technology is compared with a product, or an architectural style with a protocol specification and its implementations. Pseudo-alternatives sometimes can be found too, but do not help.

## Chosen Option.  
*mandatory*  
Here, the chosen option is referred to by its title. A justification should be given as well: {name of option 1} because {justification}. Some examples of justifications are: it is the only option that meets a certain k.o. criterion/decision driver; it resolves a particular force; it comes out best when comparing options. 

## Consequences.  
*mandatory*  
This section discusses how problem and solution space look like after the decision is made (and enforced).

Positive and negative consequences are listed as “Good, because …” and “Bad, because …”, respectively. An example for a positive consequence is an improvement of a desired quality. A negative consequence might be extra effort or risk during implementation.

## Validation.  
*optional*  
This optional section describes how the implementation of/compliance with the ADR is evaluated (aka enforced), for instance by way of a review or a test. See “A Definition of Done for Architectural Decision Making” for related hints.

## Pros and Cons of the Options.  
*mandatory*  
Here, the alternatives that address the problem can be explained and analyzed more thoroughly.

The template advises to provide an example or a description of the option. Then, “Good” and “Bad” options properties are asked for. For noteworthy “Neutral” arguments, the template suggests the form

Neutral (w.r.t.), because {argument}.

## More Information.  
*optional*  
Here, one might want to provide additional evidence for the decision outcome (possibly including assumptions made) and/or document the team agreement on the decision (including the confidence level) and/or define how this decision should be realized and when it should be re-visited (the optional “Validation” section may also cover this aspect). Links to other decisions and resources might appear in this section as well.

Include a reference to the Design Doc that this ADR has originated from. While the ADR is lightweight, the design doc contains more detailed descriptions and diagrams.
"""


In [ ]:
system_prompt = f"You are an experienced software architect. Given the following context, generate an Architecture Decision Record (ADR) in Markdown format. Include the problem, decision, status, alternatives considered, pros/cons, and consequences. \
You are provided via the chat with instructions about what to write the ADR for. \
You must adhere to the structure of the following template: {adr_template}"

system_prompt += f"\n\n## Architecture document:\n{architecture_proposal}\n\n"
system_prompt += f"With this context, please chat with the user."

system_prompt += f"Crucial Rules:\
- You must allow the user to make revisions on the ADR you write.\
- You must ask the user whether the ADR is ready to be handed off, and if so, you must hand it off to the store_adr tool."


In [ ]:
system_prompt

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(chat, type="messages").launch()